In [ ]:
!pip install ../input/sacremoses/sacremoses-master/ > /dev/null

import os
import sys
import glob
import torch

sys.path.insert(0, "../input/transformers/transformers-master/")
import transformers
import numpy as np
import pandas as pd
import math

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import argparse
import collections
import datetime
import gc
import glob
import logging
import math
import operator
import os 
import pickle
import pkg_resources
import random
import re
import scipy.stats as stats
import seaborn as sns
import shutil
import sys
import time
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from contextlib import contextmanager
from collections import OrderedDict
# from nltk.stem import PorterStemmer
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from torch.nn import CrossEntropyLoss, MSELoss
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import (Dataset,DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
%load_ext autoreload
%autoreload 2
%matplotlib inline
# from tqdm import tqdm, tqdm_notebook, trange
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
# from apex import amp

In [ ]:
from torch.utils.data.distributed import DistributedSampler

try:
    from torch.utils.tensorboard import SummaryWriter
except:
    from tensorboardX import SummaryWriter

from transformers import (WEIGHTS_NAME, BertConfig, BertModel,
                                  BertForQuestionAnswering, BertTokenizer,
                                  DistilBertTokenizer, DistilBertModel)

from transformers import AdamW # , WarmupLinearSchedule
from transformers.tokenization_bert import (BasicTokenizer,
                                                    whitespace_tokenize)

SEED = 1129

def seed_everything(seed=1129):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(SEED)

In [ ]:
mispell_dict = {
    "I'd": 'I would',
    "I'll": 'I will',
    "I'm": 'I am',
    "I've": 'I have',
    "ain't": 'is not',
    "aren't": 'are not',
    "can't": 'cannot',
    'cancelled': 'canceled',
    'centre': 'center',
    'colour': 'color',
    "could've": 'could have',
    "couldn't": 'could not',
    "didn't": 'did not',
    "doesn't": 'does not',
    "don't": 'do not',
    'enxiety': 'anxiety',
    'favourite': 'favorite',
    "hadn't": 'had not',
    "hasn't": 'has not',
    "haven't": 'have not',
    "he'd": 'he would',
    "he'll": 'he will',
    "he's": 'he is',
    "here's": 'here is',
    "how's": 'how is',
    "i'd": 'i would',
    "i'll": 'i will',
    "i'm": 'i am',
    "i've": 'i have',
    "isn't": 'is not',
    "it'll": 'it will',
    "it's": 'it is',
    'labour': 'labor',
    "let's": 'let us',
    "might've": 'might have',
    "must've": 'must have',
    'organisation': 'organization',
    "she'd": 'she would',
    "she'll": 'she will',
    "she's": 'she is',
    "shouldn't": 'should not',
    "that's": 'that is',
    'theatre': 'theater',
    "there's": 'there is',
    "they'd": 'they would',
    "they'll": 'they will',
    "they're": 'they are',
    "they've": 'they have',
    'travelling': 'traveling',
    "wasn't": 'was not',
    'watsapp': 'whatsapp',
    "we'd": 'we would',
    "we'll": 'we will',
    "we're": 'we are',
    "we've": 'we have',
    "weren't": 'were not',
    "what's": 'what is',
    "where's": 'where is',
    "who'll": 'who will',
    "who's": 'who is',
    "who've": 'who have',
    "won't": 'will not',
    "would've": 'would have',
    "wouldn't": 'would not',
    "you'd": 'you would',
    "you'll": 'you will',
    "you're": 'you are',
    "you've": 'you have',
    '，': ',',
    '／': '/',
    '？': '?'
}


puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\n', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]


def clean_puncts(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_text(x):
    x = str(x)        
    x = x.lower()
    x = x.replace('\n', '') # 改行削除
    x = x.replace('\t', '') # タブ削除
    x = x.replace('\r', '')
    x = re.sub(re.compile(r'[!-\/:-@[-`{-~]'), ' ', x) 
    x = re.sub(r'\[math\]', ' LaTex math ', x) # LaTex削除
    x = re.sub(r'\[\/math\]', ' LaTex math ', x) # LaTex削除
    x = re.sub(r'\\', ' LaTex ', x) # LaTex削除   
    x = re.sub(' +', ' ', x)
    return x

def clean_text_simple(x):
    x = str(x)        
    x = x.lower()
    x = x.replace('\n', '') # 改行削除
    x = x.replace('\t', '') # タブ削除
    x = x.replace('\r', '')
    x = x.replace('?', '.')
    x = x.replace('!', '.')
    x = re.sub(' +', ' ', x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(['\s*'.join(key) 
                                               for key in mispell_dict.keys()]))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[re.sub('\s', '', match.group(0))]
    return mispellings_re.sub(replace, text)


def preprocess(data):
    data = data.progress_apply(lambda x: clean_puncts(x))
    data = data.progress_apply(lambda x: replace_typical_misspell(x))
    data = data.progress_apply(lambda x: clean_text(x))
    return data

In [ ]:
train = pd.read_csv("../input/google-quest-challenge/train.csv")
test = pd.read_csv("../input/google-quest-challenge/test.csv")

In [ ]:
from urllib.parse import urlparse

# ===================
# create url features
# ===================

find = re.compile(r"^[^.]*")

train['netloc'] = train['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
test['netloc'] = test['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])

features = ['netloc', 'category']
merged = pd.concat([train[features], test[features]])
ohe = OneHotEncoder()
ohe.fit(merged)

netloc_category_features_train = ohe.transform(train[features]).toarray()
netloc_category_features_test = ohe.transform(test[features]).toarray()

print(netloc_category_features_train.shape, netloc_category_features_test.shape)

In [ ]:
def get_sentence_features(train, col):
    train[col + '_num_chars'] = train[col].apply(len)
    train[col + '_num_capitals'] = train[col].apply(lambda x: sum(1 for c in x if c.isupper()))
    train[col + '_caps_vs_length'] = train.apply(lambda row: row[col + '_num_chars'] / (row[col + '_num_capitals']+1e-5), axis=1)
    train[col + '_num_exclamation_marks'] = train[col].apply(lambda x: x.count('!'))
    train[col + '_num_question_marks'] = train[col].apply(lambda x: x.count('?'))
    train[col + '_num_punctuation'] = train[col].apply(lambda x: sum(x.count(w) for w in '.,;:'))
    train[col + '_num_symbols'] = train[col].apply(lambda x: sum(x.count(w) for w in '*&$%'))
    train[col + '_num_words'] = train[col].apply(lambda x: len(x.split()))
    train[col + '_num_unique_words'] = train[col].apply(lambda comment: len(set(w for w in comment.split())))
    train[col + '_words_vs_unique'] = train[col + '_num_unique_words'] / train[col + '_num_words'] 
    return train


for col in tqdm(['question_body', 'question_title', 'answer']):
    train = get_sentence_features(train, col)
    test = get_sentence_features(test, col)

In [ ]:
all_num_features = [
       'question_body_num_chars',
       'question_body_num_capitals', 'question_body_caps_vs_length',
       'question_body_num_exclamation_marks',
       'question_body_num_question_marks', 'question_body_num_punctuation',
       'question_body_num_symbols', 'question_body_num_words',
       'question_body_num_unique_words', 'question_body_words_vs_unique',
       'question_title_num_chars', 'question_title_num_capitals',
       'question_title_caps_vs_length', 'question_title_num_exclamation_marks',
       'question_title_num_question_marks', 'question_title_num_punctuation',
       'question_title_num_symbols', 'question_title_num_words',
       'question_title_num_unique_words', 'question_title_words_vs_unique',
       'answer_num_chars', 'answer_num_capitals', 'answer_caps_vs_length',
       'answer_num_exclamation_marks', 'answer_num_question_marks',
       'answer_num_punctuation', 'answer_num_symbols', 'answer_num_words',
       'answer_num_unique_words', 'answer_words_vs_unique'
]

for feature in tqdm(all_num_features):
    scaler = StandardScaler()
    scaler.fit(train[feature].append(test[feature]).astype(np.float64).values[:, np.newaxis])
    
    train[feature + '_scaled'] = scaler.transform(train[feature].astype(np.float64).values[:, np.newaxis])
    test[feature + '_scaled'] = scaler.transform(test[feature].astype(np.float64).values[:, np.newaxis])
    
    train[feature + '_scaled'] = np.nan_to_num(train[feature + '_scaled'], -99)
    test[feature + '_scaled'] = np.nan_to_num(test[feature + '_scaled'], -99)
    
    del train[feature], test[feature]
    gc.collect()

In [ ]:
train['question_title'] = preprocess(train['question_title'])
train['question_body'] = preprocess(train['question_body'])
train['answer'] = preprocess(train['answer'])
test['question_title'] = preprocess(test['question_title'])
test['question_body'] = preprocess(test['question_body'])
test['answer'] = preprocess(test['answer'])

In [ ]:
train.head()

In [ ]:
target_cols = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']

In [ ]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        

def fetch_vectors(string_list, batch_size=64):
    # inspired by https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
    DEVICE = torch.device("cuda")
    tokenizer = transformers.DistilBertTokenizer.from_pretrained("../input/distilbertbaseuncased/")
    model = transformers.DistilBertModel.from_pretrained("../input/distilbertbaseuncased/")
    model.to(DEVICE)

    fin_features = []
    for data in tqdm(chunks(string_list, batch_size)):
        tokenized = []
        for x in data:
            x = " ".join(x.strip().split()[:300])
            tok = tokenizer.encode(x, add_special_tokens=True)
            tokenized.append(tok[:512])

        max_len = 512
        padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized])
        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded).to(DEVICE)
        attention_mask = torch.tensor(attention_mask).to(DEVICE)

        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask)

        features = last_hidden_states[0][:, 0, :].cpu().numpy()
        fin_features.append(features)

    fin_features = np.vstack(fin_features)
    return fin_features

In [ ]:
train_question_body_dense = fetch_vectors(train['question_body'].values)
train_question_title_dense = fetch_vectors(train['question_title'].values)
train_answer_dense = fetch_vectors(train['answer'].values)
train_category_dense = fetch_vectors(train['category'].values)
train_netloc_dense = fetch_vectors(train['netloc'].values)

test_question_body_dense = fetch_vectors(test['question_body'].values)
test_question_title_dense = fetch_vectors(test['question_title'].values)
test_answer_dense = fetch_vectors(test['answer'].values)
test_category_dense = fetch_vectors(test['category'].values)
test_netloc_dense = fetch_vectors(test['netloc'].values)

In [ ]:
train_question_body_dense[0][:3], test_question_body_dense[0][:3], \
train_question_title_dense[0][:3], test_question_title_dense[0][:3], \
train_answer_dense[0][:3], test_answer_dense[0][:3], \
train_category_dense[0][:3], test_category_dense[0][:3], \
train_netloc_dense[0][:3], test_netloc_dense[0][:3], \

In [ ]:
import scipy as sp
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_extraction.text import _document_frequency
from sklearn.pipeline import make_pipeline, make_union


class BM25Transformer(BaseEstimator, TransformerMixin):
    """
    Parameters
    ----------
    use_idf : boolean, optional (default=True)
    k1 : float, optional (default=2.0)
    b  : float, optional (default=0.75)
    References
    ----------
    Okapi BM25: a non-binary model - Introduction to Information Retrieval
    http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html
    """
    def __init__(self, use_idf=True, k1=2.0, b=0.75):
        self.use_idf = use_idf
        self.k1 = k1
        self.b = b

    def fit(self, X):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features] document-term matrix
        """
        if not sp.sparse.issparse(X):
            X = sp.sparse.csc_matrix(X)
        if self.use_idf:
            n_samples, n_features = X.shape
            df = _document_frequency(X)
            idf = np.log((n_samples - df + 0.5) / (df + 0.5))
            self._idf_diag = sp.sparse.spdiags(idf, diags=0, m=n_features, n=n_features)

        doc_len = X.sum(axis=1)
        self._average_document_len = np.average(doc_len)

        return self

    def transform(self, X, copy=True):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features] document-term matrix
        copy : boolean, optional (default=True)
        """
        if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
            # preserve float family dtype
            X = sp.sparse.csr_matrix(X, copy=copy)
        else:
            # convert counts or binary occurrences to floats
            X = sp.sparse.csr_matrix(X, dtype=np.float, copy=copy)

        n_samples, n_features = X.shape

        # Document length (number of terms) in each row
        # Shape is (n_samples, 1)
        doc_len = X.sum(axis=1)
        # Number of non-zero elements in each row
        # Shape is (n_samples, )
        sz = X.indptr[1:] - X.indptr[0:-1]

        # In each row, repeat `doc_len` for `sz` times
        # Shape is (sum(sz), )
        # Example
        # -------
        # dl = [4, 5, 6]
        # sz = [1, 2, 3]
        # rep = [4, 5, 5, 6, 6, 6]
        rep = np.repeat(np.asarray(doc_len), sz)

        # Compute BM25 score only for non-zero elements
        nom = self.k1 + 1
        denom = X.data + self.k1 * (1 - self.b + self.b * rep / self._average_document_len)
        data = X.data * nom / denom

        X = sp.sparse.csr_matrix((data, X.indices, X.indptr), shape=X.shape)

        if self.use_idf:
            check_is_fitted(self, '_idf_diag', 'idf vector is not fitted')

            expected_n_features = self._idf_diag.shape[0]
            if n_features != expected_n_features:
                raise ValueError("Input has n_features=%d while the model"
                                 " has been trained with n_features=%d" % (
                                     n_features, expected_n_features))
            X = X * self._idf_diag

        return X

In [ ]:
len_train = len(train)
n_components = 50


all_data = pd.concat([train, test], sort=True)

vectorizer = make_pipeline(
                TfidfVectorizer(ngram_range=(1, 3)),
                make_union(
                    TruncatedSVD(n_components=n_components, random_state=SEED),
                    make_pipeline(
                        BM25Transformer(use_idf=True, k1=2.0, b=0.75),
                        TruncatedSVD(n_components=n_components, random_state=SEED)
                    ),
                    n_jobs=1,
                ),
             )
all_question_title = vectorizer.fit_transform(all_data["question_title"]).astype(np.float32)
all_question_body = vectorizer.fit_transform(all_data["question_body"]).astype(np.float32)
all_answer = vectorizer.fit_transform(all_data["answer"]).astype(np.float32)

question_title = all_question_title[:len_train]
question_title_test = all_question_title[len_train:]
question_body = all_question_body[:len_train]
question_body_test = all_question_body[len_train:]
answer = all_answer[:len_train]
answer_test = all_answer[len_train:]

In [ ]:
question_title.shape, question_title_test.shape, \
question_body.shape, question_body_test.shape, \
answer.shape, answer_test.shape

In [ ]:
l2_dist = lambda x, y: np.power(x - y, 2).sum(axis=1)

cos_dist = lambda x, y: (x*y).sum(axis=1)


dist_features_train = np.array([
    l2_dist(train_question_body_dense, train_answer_dense),
    l2_dist(train_question_title_dense, train_answer_dense),
    l2_dist(train_question_body_dense, train_question_title_dense),
    cos_dist(train_question_body_dense, train_question_title_dense),
    cos_dist(train_question_body_dense, train_question_title_dense),
    cos_dist(train_question_body_dense, train_question_title_dense),
]).T

dist_features_test = np.array([
    l2_dist(test_question_body_dense, test_answer_dense),
    l2_dist(test_question_title_dense, test_answer_dense),
    l2_dist(test_question_body_dense, test_question_title_dense),
    cos_dist(test_question_body_dense, test_question_title_dense),
    cos_dist(test_question_body_dense, test_question_title_dense),
    cos_dist(test_question_body_dense, test_question_title_dense),
]).T

dist_features_train.shape, dist_features_test.shape

In [ ]:
train_features = np.concatenate([question_title, question_body, answer,
                                 np.vstack(train_question_body_dense),
                                 np.vstack(train_question_title_dense),
                                 np.vstack(train_answer_dense),
                                 np.vstack(train_category_dense),
                                 np.vstack(train_netloc_dense),
                                 netloc_category_features_train,
                                 np.hstack([train[[x + '_scaled' for x in all_num_features]].values]),
                                 dist_features_train,
                               ], axis = 1)
test_features = np.concatenate([question_title_test, question_body_test, answer_test,
                                np.vstack(test_question_body_dense),
                                np.vstack(test_question_title_dense),
                                np.vstack(test_answer_dense),
                                np.vstack(test_category_dense),
                                np.vstack(test_netloc_dense),
                                netloc_category_features_test,
                                np.hstack([test[[x + '_scaled' for x in all_num_features]].values]),
                                dist_features_test,
                               ], axis = 1)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from scipy.stats import spearmanr

num_folds = 5
fold_scores = []
kf = KFold(n_splits = num_folds, shuffle = True, random_state = 42)
test_preds = np.zeros((len(test_features), len(target_cols)))
for train_index, val_index in kf.split(train_features):
    train_X = train_features[train_index, :]
    train_y = train[target_cols].iloc[train_index]
    
    val_X = train_features[val_index, :]
    val_y = train[target_cols].iloc[val_index]
    
    model = Sequential([
        Dense(2048, input_shape=(train_features.shape[1],)),
        Activation('relu'),
        Dense(1024),
        Activation('relu'),
        Dense(len(target_cols)),
        Activation('sigmoid'),
    ])
    
    es = EarlyStopping(monitor='val_loss', min_delta=0, patience=6, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy')
    
    model.fit(np.asarray(train_X), np.asarray(train_y), epochs = 50, validation_data=(np.asarray(val_X), np.asarray(val_y)), callbacks = [es])
    preds = model.predict(val_X)
    overall_score = 0
    for col_index, col in enumerate(target_cols):
        overall_score += spearmanr(preds[:, col_index], val_y[col].values).correlation/len(target_cols)
        print(col, spearmanr(preds[:, col_index], val_y[col].values).correlation)
    fold_scores.append(overall_score)
    print(overall_score)

    test_preds += model.predict(test_features)/num_folds
    
print(fold_scores)

In [ ]:
sub = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")

In [ ]:
sub.shape

In [ ]:
for col_index, col in enumerate(target_cols):
    sub[col] = test_preds[:, col_index]

In [ ]:
sub.to_csv("submission.csv", index = False)